In [24]:
import os
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from IPython.display import clear_output

%matplotlib inline

In [34]:
DF_PATH = "/Users/dutking/LOCAL/Pathologies_ArtDental_g/Utkin Dmitry/DF_semifinal.csv"

DATA_DIR = "/Users/dutking/Yandex.Disk-dutking.localized/ИИ"

PROJECTIONS = ["Аксиальная", "Сагитальная", "Корональная"]
AXIAL_SUB = ["Центральное", "Латеральное", "Медиальное"]
SAGITTAL_SUB = ["Центральное", "Компрессия", "Дистракция", "Дистальное", "Мезиальное"]
CORONAL_SUB = ["Центральное", "Компрессия", "Дистракция", "Латеральное", "Медиальное"]

AXIAL_METRICS = ["anterior", "posterior", "medial", "lateral"]
SAGITTAL_METRICS = ["anterior", "superior", "posterior"]
CORONAL_METRICS = ["medial", "lateral", "superior"]

CURRENT_SUB = AXIAL_SUB

In [29]:
img_data = list(os.walk(DATA_DIR))

In [38]:
df = pd.read_csv(DF_PATH, index_col="id")
df["comment"].fillna("", inplace=True)
df.head(3)

,name,comment,axial.left.anterior,axial.left.posterior,axial.left.medial,axial.left.lateral,axial.right.anterior,axial.right.posterior,axial.right.medial,axial.right.lateral,...,issue.medial.left,issue.medial.right,issue.compression.left,issue.compression.right,issue.distraction.left,issue.distraction.right,issue.distal.left,issue.distal.right,issue.mesial.left,issue.mesial.right
id,,,,,,,,,,,,,,,,,,,,,
1,Останькович В.В.,,-1.0,-1.0,3.1,1.6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
2,Самойлова Е.Г. 29.12.2021,,-1.0,-1.0,1.7,2.3,-1.0,-1.0,2.0,2.3,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
3,Авакимова М.Н.,,-1.0,-1.0,8.4,2.4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [32]:
def get_issue(path):
    issue_name = path.split("/")[-1].split(" ")[1].split(".")[0]
    match issue_name:
        case "Центральное":
            issue_id = "normal"
        case "Латеральное":
            issue_id = "lateral"
        case "Медиальное":
            issue_id = "medial"
        case "Компрессия":
            issue_id = "compression"
        case "Дистракция":
            issue_id = "distraction"
        case "Дистальное":
            issue_id = "distal"
        case "Мезиальное":
            issue_id = "mesial"

    return issue_name, issue_id


def get_projection(path):
    projection_name = path.split("/")[-2].split(" ")[-2]
    match projection_name:
        case "Аксиальная":
            projection_id = "axial"
        case "Сагитальная":
            projection_id = "sagittal"
        case "Корональная":
            projection_id = "coronal"

    return projection_name, projection_id


def get_side(filename):
    if "Л" in filename:
        return "Слева", "left"
    elif "П" in filename:
        return "Справа", "right"


def get_metrics(projection_id):
    match projection_id:
        case "axial":
            return AXIAL_METRICS
        case "sagittal":
            return SAGITTAL_METRICS
        case "coronal":
            return CORONAL_METRICS


def input_metrics(projection_id):
    match projection_id:
        case "axial":
            data = input((" ").join(AXIAL_METRICS))
        case "sagittal":
            data = input((" ").join(SAGITTAL_METRICS))
        case "coronal":
            data = input((" ").join(CORONAL_METRICS))
    if data == "":
        return None
    try:
        result = list(map(float, data.strip().split(" ")))
    except ValueError:
        print("Необходимо ввести числа через пробел.")
        return input_metrics(projection_id)
    return result


def set_data(df, user_id, projection_id, side_id, input_data):
    metrics = get_metrics(projection_id)
    for metric, value in zip(metrics, input_data):
        df.loc[user_id, f"{projection_id}.{side_id}.{metric}"] = value


def set_issue(df, user_id, issue_id, side_id):
    if issue_id != "normal":
        df.loc[user_id, f"issue.{issue_id}.{side_id}"] = 1


def set_comment(df, user_id):
    comment = input("Комментарий: ")
    if comment == "":
        return

    if comment.startswith("+"):
        if df.loc[user_id, "comment"]:
            df.loc[user_id, "comment"] += comment[1:]
        else:
            df.loc[user_id, "comment"] = comment[1:]
        return

    df.loc[user_id, "comment"] = comment

In [16]:
bad_ids = [
    107,
    121,
    145,
    151,
    162,
    173,
    175,
    176,
    183,
    189,
    199,
    222,
    244,
    245,
    246,
    269,
    278,
]

In [33]:
for i in df.index:
    if i > 0:
        user_id = i
        user_name = df.loc[i, "name"]
        img_id_1 = f"К{user_id}Л"
        img_id_2 = f"К{user_id}П"
        for root, dirs, files in img_data:
            for file in list(sorted(files)):
                if file.startswith(img_id_1) or file.startswith(img_id_2):
                    projection_name, projection_id = get_projection(root)
                    issue_name, issue_id = get_issue(root)
                    side_name, side_id = get_side(file)
                    metrics = get_metrics(projection_id)
                    print(user_id, user_name)
                    print(
                        f"{file}: {projection_name} проекция | {issue_name} смещение | {side_name}"
                    )
                    print(f"Комментарий: {df.loc[user_id, 'comment']}")
                    print("Текущие значения:")
                    current_metrics = df.loc[
                        user_id,
                        [f"{projection_id}.{side_id}.{metric}" for metric in metrics],
                    ]
                    print([f"{k}: {v}" for (k, v) in zip(metrics, current_metrics)])
                    image = mpimg.imread(os.path.join(root, file))
                    fig = plt.figure(figsize=(12, 12))
                    plt.imshow(image)
                    plt.show()
                    set_issue(df, user_id, issue_id, side_id)
                    data = input_metrics(projection_id)
                    if data is not None:
                        set_data(df, user_id, projection_id, issue_id, side_id, data)
                    set_comment(df, user_id)
                    df.to_csv("DF_semifinal.csv")
                    clear_output(wait=True)

1 Останькович В.В.
К1Пк.jpg: Корональная проекция | Латеральное смещение | Справа
Комментарий: 
Текущие значения:
1 Останькович В.В.
К1Лк.jpg: Корональная проекция | Центральное смещение | Слева
Комментарий: 
Текущие значения:
1 Останькович В.В.
К1Лс.jpg: Сагитальная проекция | Дистальное смещение | Слева
Комментарий: 
Текущие значения:
1 Останькович В.В.
К1Пс.jpg: Сагитальная проекция | Дистальное смещение | Справа
Комментарий: 
Текущие значения:
1 Останькович В.В.
К1Ла.jpg: Аксиальная проекция | Латеральное смещение | Слева
Комментарий: 
Текущие значения:
2 Самойлова Е.Г. 29.12.2021
К2Лк.jpg: Корональная проекция | Центральное смещение | Слева
Комментарий: 
Текущие значения:
2 Самойлова Е.Г. 29.12.2021
К2Пк.jpg: Корональная проекция | Центральное смещение | Справа
Комментарий: 
Текущие значения:
2 Самойлова Е.Г. 29.12.2021
К2Лс.jpg: Сагитальная проекция | Дистальное смещение | Слева
Комментарий: 
Текущие значения:
2 Самойлова Е.Г. 29.12.2021
К2Пс.jpg: Сагитальная проекция | Дистально